# Введение в регрессионный анализ

## Практикум 4. Модели с фиктивными переменными

*Алла Тамбовцева*

### Подготовка к работе: импорт необходимых библиотек

Импортируем библиотеку `pandas` для загрузки и обработки данных и функцию `ols()` из `statsmodels` для построения линейных моделей.

In [2]:
import pandas as pd
from statsmodels.formula.api import ols

### Сюжет 1: модель с фиктивными переменными и выбор базовой категории

Загрузим данные с результатами весьма специфического исследования роста зубов у морских свинок (отвлечемся от политологии и посмотрим на модель из семинарского листочка):

In [3]:
df = pd.read_csv("Tooth.csv")
df.head()

,Unnamed: 0,Unnamed: 0.1,len,supp,dose
0,0,1,4.2,VC,Low
1,1,2,11.5,VC,Low
2,2,3,7.3,VC,Low
3,3,4,5.8,VC,Low
4,4,5,6.4,VC,Low


Переменные в датафрейме:
    
* `len`: длина зубов (в мм);
* `supp`: в каком виде морским свинкам давали витамин С (`VC` – аскорбиновая кислота, `OJ` – апельсиновый сок);
* `dose`: группа в эксперименте (`Low` – группа, которой давали маленькую дозировку витамина, `Medium` – среднюю, `High` – высокую).

Итак, нас интересует, есть ли связь между дозировкой витамина С и ростом зубов у морских свинок. Построим линейную модель, где зависимой переменной выступает длина зубов, а независимой – название экспериментальной группы:

In [4]:
ols("len ~ dose", df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    len   R-squared:                       0.703
Model:                            OLS   Adj. R-squared:                  0.692
Method:                 Least Squares   F-statistic:                     67.42
Date:                Tue, 17 Oct 2023   Prob (F-statistic):           9.53e-16
Time:                        02:16:45   Log-Likelihood:                -170.30
No. Observations:                  60   AIC:                             346.6
Df Residuals:                      57   BIC:                             352.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         26.1000      0.949     27.515      0.000      24.201      27.999
dose[T.Low]      -15.4950      1.341    -11.551      0.000     -18.181     -12.809
dose[T.Medium]    -6.3650      1.341     -4.745      0.000      -9.051      -3.679
==============================================================================
Omnibus:                        2.934   Durbin-Watson:                   1.446
Prob(Omnibus):                  0.231   Jarque-Bera (JB):                2.812
Skew:                           0.475   Prob(JB):                        0.245
Kurtosis:                       2.527   Cond. No.                         3.73
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Так как показатель `dose` является текстовым, категориальным (уникальные значения = категории), он разбивается на набор бинарных **дамми-переменных** или **фиктивных переменных**:

* `dose[T.High]`: значение `High` или нет, 1 или 0 (`T.` от `True`);
* `dose[T.Medium]`: значение `Medium` или нет, 1 или 0;
* `dose[T.Low]`: значение `Low` или нет, 1 или 0.

Чтобы информация не дублировалась и не портила модель, в нее добавляются не все фиктивные переменные, а на одну меньше, то есть одна переменная «выбрасывается». Категория, которая соответствует «выброшенной» переменной является **базовой**, референтной, с ней сравниваются все остальные группы. 

По умолчанию Python выбирает в качестве базовой категории ту, которая идет первой по алфавиту. В нашем случае базовой категорией является группа `High`, так как в выдаче с моделью не хватает именно ее. Давайте воспроизведем модель из семинарского листочка и в качестве базовой категории выберем контрольную группу `Low`.

Как это сделать? Учесть в рамках функции `ols()` это не получится, нужно перекодировать данные. Сделаем это при помощи функции `Categorical()`: 

* на первом месте указываем название столбца, который перекодируем;
* на втором месте – список значений в том порядке, который нас устраивает;
* на третьем месте – аргумент `ordered=True`, чтобы зафиксировать порядок следования.

In [5]:
df["dose_sorted"] = pd.Categorical(df["dose"], 
                                   categories = ["Low", "Medium", "High"], ordered=True)

In [6]:
# визуально все одинаково, только в случае dose_sorted 
# Python понимает, что Low < Medium < High

df.head()

,Unnamed: 0,Unnamed: 0.1,len,supp,dose,dose_sorted
0,0,1,4.2,VC,Low,Low
1,1,2,11.5,VC,Low,Low
2,2,3,7.3,VC,Low,Low
3,3,4,5.8,VC,Low,Low
4,4,5,6.4,VC,Low,Low


Готово! Теперь, если мы оценим модель еще раз, в качестве базовой категории Python выберет `Low`, поскольку она является первой в том порядке, который мы установили. 

In [7]:
print(ols("len ~ dose_sorted", df).fit().summary())

                            OLS Regression Results                            
Dep. Variable:                    len   R-squared:                       0.703
Model:                            OLS   Adj. R-squared:                  0.692
Method:                 Least Squares   F-statistic:                     67.42
Date:                Tue, 17 Oct 2023   Prob (F-statistic):           9.53e-16
Time:                        02:16:55   Log-Likelihood:                -170.30
No. Observations:                  60   AIC:                             346.6
Df Residuals:                      57   BIC:                             352.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                10.60

Приступим к интерпретации. Запишем уравнение модели:

$$
\widehat{\text{len}}_i = 10.61 + 9.13 \times \text{Medium}_i + 15.5 \times \text{High}_i.
$$
    
    

Итак: 
    
* `Intercept`: среднее ожидаемое значение зависимой переменной в случае, когда все независимые равны 0; если `Medium = 0` и `High = 0`, то `Low = 1` по определению модели, значит, это среднее значение `len` в базовой категории, то есть средняя длина зубов (10.61 мм) морских свинок в контрольной группе, где грызунам давали витамин C в маленькой дозировке.

* Коэффициент при `Medium` показывает, на сколько, в среднем, отличаются значения зависимой переменной в базовой категории `Low` и категории `Medium`. Можем заключить, что средняя длина зубов в группе свинок, которым давали витамин C в средней дозировке, на 9.13 мм больше, чем в контрольной группе с маленькой дозировкой.

* Коэффициент при `High` показывает, на сколько, в среднем, отличаются значения зависимой переменной в базовой категории `Low` и категории `High`. Можем заключить, что средняя длина зубов в группе свинок, которым давали витамин C в большой дозировке, на 15.5 мм больше, чем в контрольной группе с маленькой дозировкой.

При этом, все различия в группах являются статистическими значимыми, p-value везде примерно 0, значит, на 5%-ном уровне значимости мы можем заключить, что оценки коэффициентов значимо отличны от 0.

Модель при этом обладает достаточно высокой прогностической силой, $R^2$ равен 0.7, значит, модель объясняет 70% дисперсии зависимой переменной. На 70% изменчивость значений длины зубов морских свинок можно объяснить различиями в дозировке витамина C.

### Сюжет 2. Модель с фиктивными переменными и изменение типа независимой переменной

Давайте частично воспроизведем модель из статьи **E.Patacchini et al "Information Transmission in a Social Network: A Field Experiment"** и выясним, зависит ли успешность групповой работы от типа устройства сети сообщества, при условии, что тип сети определяет характер передачи информации, которая приближает группу к выигрышу. 

В этой статье описывается экспериментальная игра, в которой участвовали ученики одной школы Италии. Школьников случайным образом распределяли на группы по 5 человек и давали каждому индивидуальное задание – угадать все цвета элементов одежды персонажа (шапка, рубашка, брюки, перчатки, ботинки), который показывался им на картинке в приложении. Каждому участнику был достоверно известен только цвет какого-то одного элемента одежды. Идея понятна: в идеальной ситуации в группе из пяти человек каждый может поделиться своей частью информации с другим, узнать все пять элементов одежды и остаться в выигрыше. Но все не так просто. Школьники не знают, с кем они находятся в группе, эта информация им доступна лишь частично: кто-то знает имена трех членов группы, кто-то – только двух, кто-то – только одного. Никакого чата в игре нет и участники не могут его создать, они могут лишь вживую, зная имя человека, обменяться с ним информацией.

Таким образом, группы из пяти человек устроены по-разному, по разным типам сети. 

* Первый тип сети: довольно базовый, участники знают одного-двух участников группы, при этом есть наиболее осведомленный член группы, который знает трех участников.

* Второй тип сети: явное преимущество у одного члена группы, он знает имена всех четырех участников, остальные – лишь двоих.

* Третий тип сети: более равномерное число связей, все участники, кроме одного, знают имена двух-трех участников.

За угадывания цветов игроки получают баллы, попытки с неправильными ответами штрафуются. Участники, набравшие наибольшее число очков, могли выиграть суммы от 0 до 15 евро, выигрыш вручался в виде подарочных карт от Amazon на определенную сумму. 

Итого: интересно посмотреть, каким образом тип сети (а, следовательно, и конфигурация системы передачи информации) сказывается на групповом выигрыше. Если смотреть только на устройство сети, кажется, что вторая сеть, где есть участник, знающий всех, идеальна, потому что он будет центральным «узлом» передачи информации для всех игроков. Но в реальности, где рациональный выбор все же существует, все может оказаться иначе: центральному игроку, знающему всех, может быть невыгодно служить источником информации для остальных, так как передавая правдивую информацию он, в том числе, повышает шансы других угадать больше ответов и получить выигрыш.

Загрузим данные с результатами эксперимента:

In [9]:
data = pd.read_csv("networks.csv")
data.head()

,Unnamed: 0,phase,character,idusers,idnetwork,hint_cloth,hint_col,hat_guess,shirt_guess,pant_guess,...,score_glove,score,num_corrects,num_mistakes,grey,winner,grp_act_num,grp_num_corrects,grp_num_mistakes,pos
0,1,1.0,Freddie,358,1,cappello,nero,9.0,5.0,4.0,...,10.0,50.0,5.0,0.0,0.0,1.0,5.0,25.0,0.0,a
1,3,1.0,Piergiorgio,603,1,guanti,bianco,10.0,10.0,2.0,...,10.0,20.0,2.0,0.0,3.0,1.0,5.0,11.0,2.0,a
2,8,1.0,Marshall,447,1,cappello,bianco,8.0,2.0,2.0,...,10.0,50.0,5.0,0.0,0.0,1.0,5.0,21.0,1.0,a
3,14,1.0,Alberto,184,1,guanti,blu,4.0,5.0,7.0,...,10.0,5.0,2.0,3.0,0.0,0.0,5.0,16.0,8.0,a
4,18,1.0,Teseo,1417,1,cappello,arancione,2.0,3.0,1.0,...,10.0,35.0,4.0,1.0,0.0,1.0,5.0,15.0,4.0,a


Итак, построим линейную модель, где зависимая переменная – число правильных ответов на группу, а независимая – тип сети:

In [10]:
mod1 = ols("grp_num_corrects ~ idnetwork", data).fit()
print(mod1.summary())

                            OLS Regression Results                            
Dep. Variable:       grp_num_corrects   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:                     17.05
Date:                Tue, 17 Oct 2023   Prob (F-statistic):           4.12e-05
Time:                        02:19:56   Log-Likelihood:                -2064.8
No. Observations:                 660   AIC:                             4134.
Df Residuals:                     658   BIC:                             4143.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     14.7510      0.589     25.036      0.0

Модель, которую мы только что построили, некорректная. Почему? Посмотрим на `idnetwork`. По-хорошему, у нас в модели должно быть две фиктивные переменные, одна для сравнения второго типа с первым, вторая – для сравнения третьего типа с первым. Здесь такого не происходит, потому что тип столбца `idnetwork` целочисленный, и Python воспринимает его как полноценную числовую переменную. Как это поправить? Можно изменить тип переменной в датафрейме глобально, а можно просто учесть это в формуле `ols()`:

In [11]:
# C() – от Categorical, то есть
# смотри на переменную как на качественную, категориальную

mod1 = ols("grp_num_corrects ~ C(idnetwork)", data).fit()
print(mod1.summary())

                            OLS Regression Results                            
Dep. Variable:       grp_num_corrects   R-squared:                       0.046
Model:                            OLS   Adj. R-squared:                  0.043
Method:                 Least Squares   F-statistic:                     15.89
Date:                Tue, 17 Oct 2023   Prob (F-statistic):           1.82e-07
Time:                        02:20:00   Log-Likelihood:                -2057.7
No. Observations:                 660   AIC:                             4121.
Df Residuals:                     657   BIC:                             4135.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            16.4615      0.39

Теперь все нормально. Проинтерпретируем!

* `Intercept`: константа модели, содержательно это среднее значение зависимой переменной в базовой категории. Базовая категория здесь – это группы, устроенные по первому типу сети, с `idnetwork` равной 1, так как эта категория «выкинута» из выдачи. То есть, мы можем сказать, что среднее общегрупповое число верных ответов в группах, устроенных по типу первой сети, примерно равно 16. 

* `C(idnetwork)[T.2]`: коэффициент при втором типе сети, показывает разницу средних значений зависимой переменной в этой категории и базовой категории. Можем сказать, что среднее общегрупповое число верных ответов в группах, устроенных по типу второй сети, ниже на 0.75 (то есть примерно на 1), чем среднее число верных ответов в группах, устроенных по типу первой сети. Однако этот коэффициент является незначимым на любом разумном уровне значимости (и на 5%, и на 10%) в силу высокого p-value 0.169, поэтому интерпретировать его особого смысла нет. 

* `C(idnetwork)[T.3]`: коэффициент при третьем типе сети, показывает разницу средних значений зависимой переменной в этой категории и базовой категории. Можем сказать, что среднее общегрупповое число верных ответов в группах, устроенных по типу третьей сети, на 2 выше, чем среднее число верных ответов в группах, устроенных по типу первой сети. Оценка коэффициента в данном случае статистически значима на 5%-ном уровне значимости (и на 1%-ном тоже), поэтому различия точно есть. Выходит, третий тип сети самый выгодный, если нас интересует максимизация выигрыша всей группы.